In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ase.visualize import view
from ase.io import read
from ase.build import graphene
import seaborn as sb

sb.set_theme("notebook", "darkgrid")

In [ ]:
## Use broadcasting
def Greens_broadcast(energy, h, h_s, V, eta=1e-4, tol=1e-6, max_iter=30, **kwargs):
    """
    Compute the Green's function for a semi-infinite chain using numpy broadcasting.

    This function calculates the bulk and surface Green's functions for a system with 
    a semi-infinite chain, using iterative convergence. Broadcasting is utilized for 
    efficient matrix operations.

    Parameters
    ----------
    energy : ndarray of shape `(K,)`
        Array of energy values at which the Green's function is evaluated.
    h : ndarray of shape `(N, N)`
        On-site matrix for the bulk region of the semi-infinite chain.
    h_s : ndarray of shape `(N, N)`
        On-site matrix for the surface region (initial site of the chain).
    V : ndarray of shape `(N, N)`
        Hopping matrix that defines the coupling between adjacent sites.
    eta : float, optional
        Small imaginary component added to the energy for numerical stability, by default 1e-4.
    tol : float, optional
        Tolerance for convergence in the iterative calculation, by default 1e-6.
    max_iter : int, optional
        Maximum number of iterations allowed for convergence, by default 30.

    Returns
    -------
    G_bulk : ndarray of shape `(K, N, N)`
        Array representing the bulk Green's function at each energy point.
    G_surf : ndarray of shape `(K, N, N)`
        Array representing the surface Green's function at each energy point.

    Notes
    -----
    - Convergence is achieved when the change in the self-energy terms between iterations 
      is below the specified tolerance (`tol`).
    - The imaginary component (`eta`) ensures numerical stability but should be chosen small 
      enough to avoid affecting the results significantly.
    """
    
    # Get dimensions of input
    K = len(energy)
    N, _ = V.shape
    n_dim = len(V.shape)
    
    # Define alpha_0, beta_0
    alpha = V.transpose().copy()[None, ...]
    beta = V.copy()[None, ...]
    
    # Define epsilon_bulk_0, epsilon_surface_0
    E_bulk = h.copy()[None, ...]
    E_surf = h_s.copy()[None, ...]
    
    # Define perturbated energy of size (K, N, N)
    z = (energy + 1.j*eta)[:, *(None,)*n_dim] * np.identity(N)
    
    # Iterate parameters until convergence or max_iter reached
    for _iteration in range(max_iter):
        
        # Iterate bulk and surface greens energy
        G_bulk = np.linalg.inv(z - E_bulk)    
            
        # Iterate alpha and beta
        alpha_new = alpha @ G_bulk @ alpha
        beta_new =   beta @ G_bulk @ beta
        
        # Compute Left and Right self energies
        L_energy =  beta @ G_bulk @ alpha
        R_energy = alpha @ G_bulk @ beta
        
        # Bulk_self_energy = R_energy + L_energy
        
        # Iterate bulk and surface energy
        E_bulk_new = E_bulk + R_energy + L_energy
        E_surf_new = E_surf + R_energy
        
        # Check for convergence
        if np.max(np.abs(alpha)) < tol:
            # break if one of the values is below the tolerance level
            break
        
        # Iterate by redefining parameters
        alpha = alpha_new.copy()
        beta =  beta_new.copy()
        E_bulk = E_bulk_new.copy()
        E_surf = E_surf_new.copy()
        
    G_surf = np.linalg.inv(z - E_surf)
    return G_bulk, G_surf, E_bulk, E_surf


In [23]:
def sort_atoms(atoms):
    # Use numpy.lexsort to sort by z, then y, then x
    pos = atoms.get_positions()
    sorted_indices = np.lexsort((pos[:, 2], pos[:, 1], pos[:, 0]))
    return atoms[sorted_indices]

gr0 = graphene(formula='C2', vacuum=10.)  ## Note you can replace C2 by BN to create hexagonal BN sheets.
a1, a2, a3 = gr0.cell
A2 = a1 + 2*a2

grsq = gr0.copy()
grtmp = gr0.copy()
atom2 = grtmp[0]
atom2.position +=  (a1+a2)
grsq = grsq + atom2
grtmp = gr0.copy()
atom3 = grtmp[1]
atom3.position +=  a2
grsq = grsq + atom3
grsq.set_cell([a1,A2,a3])
# view(grsq) # before sorting by z, then y, then x.
grsq = sort_atoms(grsq)
view(grsq)

<Popen: returncode: None args: ['c:\\Users\\basti\\anaconda3\\envs\\10325\\p...>

In [25]:
def hamiltonian(xyz):
    a = 1.45 # Å
    
    # get pair-wise distance
    dist = np.linalg.norm(xyz[None, :, :] - xyz[:, None, :], axis=2)
    return np.where((dist < (a + 0.1)) & (dist > 0.1), 1, 0)

<Popen: returncode: None args: ['c:\\Users\\basti\\anaconda3\\envs\\10325\\p...>